# [실습 2] Custom Tool 만들기
학습 내용:
1. 환경 설정 및 기본 세팅
2. Custom Tool을 만드는 4가지 방법
   - @tool 데코레이터: 가장 간단한 방법
   - StructuredTool 클래스: 인자(argument)에 대한 상세한 설명이 필요할 때
   - BaseTool 클래스 상속: 가장 유연하고 확장성 있는 방법
   - LangChain Runnable(LCEL) 활용: LangChain 표현식 언어를 도구로 변환
3. Agent 생성: 직접 만든 Custom Tool을 탑재한 Agent 만들기


## 1. 환경 설정

In [1]:
# !pip install -qU langchain langchain_openai

In [1]:
import os
from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
import datetime

# LangChain 관련 기본 import
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool, BaseTool
from langchain.pydantic_v1 import BaseModel, Field
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from typing import Type

/home/elicer/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# LLM 모델을 초기화합니다.
llm = ChatOpenAI(
    model_name = "gpt-4o-mini", # e.g. "openai/gpt-4o-mini", "openai/gpt-5"
    temperature=0
)

## 2. Custom Tool을 만드는 4가지 방법
- Agent가 사용할 도구는 명확한 '이름(name)', '설명(description)',
- 그리고 '인자 스키마(args_schema)'를 가져야 합니다.
- 특히 '설명'은 Agent가 도구의 용도를 파악하는 데 결정적인 역할을 합니다.

### 2-1. @tool 데코레이터 사용
가장 직관적이고 간단하게 함수를 도구로 만드는 방법입니다.
함수의 docstring이 자동으로 도구의 설명(description)이 됩니다.

In [6]:
from langchain.tools import tool
from datetime import datetime

@tool
def get_current_time(format: str = "%Y-%m-%d %H:%M:%S") -> str:
    """
    현재 날짜와 시간을 지정된 포맷으로 반환합니다.
    사용자가 '지금 몇 시야?', '오늘 날짜 알려줘' 등 시간과 관련된 질문을 할 때 사용하세요.
    """
    return datetime.now().strftime(format)

In [7]:
# tool 데코레이터로 생성된 도구의 정보를 확인해봅시다.
print("--- @tool 데코레이터로 만든 도구 정보 ---")
print(f"이름: {get_current_time.name}")
print(f"설명: {get_current_time.description}")
print(f"인자 스키마: {get_current_time.args}")
print("-" * 20)

--- @tool 데코레이터로 만든 도구 정보 ---
이름: get_current_time
설명: 현재 날짜와 시간을 지정된 포맷으로 반환합니다.
사용자가 '지금 몇 시야?', '오늘 날짜 알려줘' 등 시간과 관련된 질문을 할 때 사용하세요.
인자 스키마: {'format': {'default': '%Y-%m-%d %H:%M:%S', 'title': 'Format', 'type': 'string'}}
--------------------


### 2-2. StructuredTool 클래스 사용
도구에 전달되는 인자(argument)에 대해 더 상세한 설명이 필요할 때 유용합니다.
Pydantic 모델을 사용하여 인자의 타입과 설명을 명확하게 정의할 수 있습니다.

In [36]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

# 계산기 함수의 인자를 정의할 Pydantic 모델
class CalculatorInput(BaseModel):
    a: int = Field(description="첫 번째 숫자")
    b: int = Field(description="두 번째 숫자")

def multiply(a: int, b: int) -> int:
    """두 수를 곱한 결과를 반환합니다."""
    return a * b

In [37]:
# StructuredTool.from_function()을 사용하여 함수로부터 도구를 생성합니다.
multiply_tool = StructuredTool.from_function(
    func=multiply,
    name="Multiplier",
    description="두 개의 숫자를 곱셈하는 계산기입니다. 곱셈이 필요할 때 사용하세요.",
    args_schema=CalculatorInput
)

In [38]:
print("--- StructuredTool로 만든 도구 정보 ---")
print(f"이름: {multiply_tool.name}")
print(f"설명: {multiply_tool.description}")
print(f"인자 스키마: {multiply_tool.args}")
print("-" * 20)

--- StructuredTool로 만든 도구 정보 ---
이름: Multiplier
설명: 두 개의 숫자를 곱셈하는 계산기입니다. 곱셈이 필요할 때 사용하세요.
인자 스키마: {'a': {'description': '첫 번째 숫자', 'title': 'A', 'type': 'integer'}, 'b': {'description': '두 번째 숫자', 'title': 'B', 'type': 'integer'}}
--------------------


### 2-3. BaseTool 클래스 상속
가장 유연한 방법으로, 클래스로 도구를 정의합니다.
동기/비동기 실행을 모두 구현하거나, 복잡한 로직을 캡슐화할 때 적합합니다.

In [39]:
# 인자 스키마 정의
class UserProfileInput(BaseModel):
    user_id: str = Field(description="정보를 조회할 사용자의 아이디")

class UserProfileTool(BaseTool):
    name: str = "get_user_profile"
    description: str = "주어진 사용자 ID에 해당하는 프로필 정보(이름, 이메일)를 조회합니다. 사용자 정보를 찾아야 할 때 사용하세요."
    args_schema: Type[BaseModel] = UserProfileInput

    def _run(self, user_id: str) -> dict:
        """도구의 동기적 실행 로직을 구현합니다."""
        # 실제 애플리케이션에서는 DB나 API를 조회하는 코드가 들어갑니다.
        # 여기서는 예시를 위해 더미 데이터를 반환합니다.
        if user_id == "student":
            return {"name": "홍길동", "email": "gildong.hong@langchain.com"}
        else:
            return {"error": "사용자를 찾을 수 없습니다."}

    async def _arun(self, user_id: str) -> dict:
        """도구의 비동기적 실행 로직을 구현합니다."""
        # 비동기 DB 조회 등의 코드를 여기에 구현할 수 있습니다.
        return self._run(user_id)

user_profile_tool = UserProfileTool()

In [40]:
print("--- BaseTool 상속으로 만든 도구 정보 ---")
print(f"이름: {user_profile_tool.name}")
print(f"설명: {user_profile_tool.description}")
print(f"인자 스키마: {user_profile_tool.args}")
print("-" * 20)

--- BaseTool 상속으로 만든 도구 정보 ---
이름: get_user_profile
설명: 주어진 사용자 ID에 해당하는 프로필 정보(이름, 이메일)를 조회합니다. 사용자 정보를 찾아야 할 때 사용하세요.
인자 스키마: {'user_id': {'description': '정보를 조회할 사용자의 아이디', 'title': 'User Id', 'type': 'string'}}
--------------------


### 2-4. LangChain Runnable(LCEL) 활용
LangChain Expression Language(LCEL)로 구성된 체인(Chain) 자체를 도구로 만들 수 있습니다.
복잡한 프롬프트와 모델 호출을 하나의 도구로 캡슐화할 때 강력합니다.

In [41]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable
from langchain.tools.render import render_text_description

# 간단한 요약 체인(Runnable)을 정의합니다.
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that summarizes text in Korean."),
    ("human", "다음 텍스트를 한 문장으로 요약해 주세요: {text_to_summarize}")
])
summarizer_chain = prompt_template | llm

# Runnable을 Tool로 변환합니다.
summarizer_tool = Tool(
    name="Text_Summarizer",
    func=summarizer_chain.invoke,
    description="주어진 텍스트를 한 문장으로 요약합니다. 긴 글을 요약해야 할 때 사용하세요.",
    # Runnable은 args_schema를 자동으로 추론하지 못하므로 직접 정의해줘야 합니다.
    # 이 경우, invoke의 인자인 'text_to_summarize'를 설명합니다.
    # 간단한 단일 문자열 인자이므로, 여기서는 생략하고 func.invoke를 사용합니다.
)

In [42]:
print("--- Runnable로 만든 도구 정보 ---")
print(f"이름: {summarizer_tool.name}")
print(f"설명: {summarizer_tool.description}")
print("-" * 20)

--- Runnable로 만든 도구 정보 ---
이름: Text_Summarizer
설명: 주어진 텍스트를 한 문장으로 요약합니다. 긴 글을 요약해야 할 때 사용하세요.
--------------------


## 3. Agent 생성 및 실행
이제 우리가 직접 만든 4개의 Custom Tool을 Agent에게 장착시켜 보겠습니다.

In [43]:
tools = [
    get_current_time,
    multiply_tool,
    user_profile_tool,
    summarizer_tool
]

In [44]:
# 프롬프트와 Agent, AgentExecutor를 생성합니다. (실습 1과 동일)
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [45]:
# Custom Tool을 활용하는 질문들을 테스트해봅니다.
print("\n--- Agent 실행 예시 1 (시간 물어보기) ---")
response1 = agent_executor.invoke({"input": "지금 몇 시야?"})
print("\n[최종 답변]:", response1["output"])


--- Agent 실행 예시 1 (시간 물어보기) ---


> Entering new AgentExecutor chain...

Invoking: `get_current_time` with `{}`


2025-09-02 05:45:23지금은 2025년 9월 2일 05시 45분입니다.

> Finished chain.

[최종 답변]: 지금은 2025년 9월 2일 05시 45분입니다.


In [46]:
print("\n--- Agent 실행 예시 2 (곱셈하기) ---")
response2 = agent_executor.invoke({"input": "12345 곱하기 54321은 얼마야?"})
print("\n[최종 답변]:", response2["output"])


--- Agent 실행 예시 2 (곱셈하기) ---


> Entering new AgentExecutor chain...

Invoking: `Multiplier` with `{'a': 12345, 'b': 54321}`


67059274512345 곱하기 54321은 670592745입니다.

> Finished chain.

[최종 답변]: 12345 곱하기 54321은 670592745입니다.


In [47]:
print("\n--- Agent 실행 예시 3 (사용자 정보 조회) ---")
response3 = agent_executor.invoke({"input": "student 사용자의 이메일 주소를 알려줘."})
print("\n[최종 답변]:", response3["output"])


--- Agent 실행 예시 3 (사용자 정보 조회) ---


> Entering new AgentExecutor chain...

Invoking: `get_user_profile` with `{'user_id': 'student'}`


{'name': '홍길동', 'email': 'gildong.hong@langchain.com'}학생 사용자의 이메일 주소는 gildong.hong@langchain.com입니다.

> Finished chain.

[최종 답변]: 학생 사용자의 이메일 주소는 gildong.hong@langchain.com입니다.


In [49]:
print("\n--- Agent 실행 예시 4 (복합 질문) ---")
# 여러 도구를 순차적으로 사용해야 하는 복잡한 질문
complex_question = "student 사용자의 이름이 뭐야? 그리고 25와 4를 곱한 값도 알려줘."
response4 = agent_executor.invoke({"input": complex_question})
print("\n[최종 답변]:", response4["output"])


--- Agent 실행 예시 4 (복합 질문) ---


> Entering new AgentExecutor chain...

Invoking: `get_user_profile` with `{'user_id': 'student'}`


{'name': '홍길동', 'email': 'gildong.hong@langchain.com'}
Invoking: `Multiplier` with `{'a': 25, 'b': 4}`


100학생 사용자의 이름은 홍길동입니다. 그리고 25와 4를 곱한 값은 100입니다.

> Finished chain.

[최종 답변]: 학생 사용자의 이름은 홍길동입니다. 그리고 25와 4를 곱한 값은 100입니다.
